1. 모든 변수를 사용하여 OLS 모델을 적합하고, 회귀계수 중 가장 큰 값은?
2. 유의미하지 않은 변수를 제거한 후 모델을 다시 적합하고, 회귀계수 중 가장 작은 변수명은?
3. 2번 모델의 R-squared 값을 계산하고 해석하세요.
4. 1번 모델에서 새로운 데이터(x1=5, x2=12, x3=10, x4=3)에 대해 y 값을 예측하세요.
5. 1번 모델에서 x1, x2, x3, x4의 상관관계를 계산하고 가장 큰 상관계수를 구하시오. (단, 자기 상관관계 제외)
6. x1과 x2만을 예측 변수로 사용하는 모델을 적합하고, 전체 모델과 R-squared 값을 구하시오.
7. 잔차(residual) 분석을 수행하고, 잔차의 표준편차를 구하시오.
8. 1번 모델에서 새로운 데이터(x1=5, x2=12, x3=10, x4=3)에 대해 y의 신뢰구간 하한(97% 신뢰수준)을 구하세요.
9. 1번 모델에서 새로운 데이터(x1=5, x2=12, x3=10, x4=3)에 대해 y의 예측구간 상한(97% 신뢰수준)을 구하세요.

In [1]:
import pandas as pd
df = pd.read_csv("/kaggle/input/bigdatacertificationkr/t3_regression_data.csv")

In [2]:
# OLS를 적합하자.
from statsmodels.formula.api import ols
model=ols('y~x1+x2+x3+x4',data=df).fit()
print(model.summary())
print('--------------')
print(model.params.max())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     1984.
Date:                Fri, 20 Jun 2025   Prob (F-statistic):           1.39e-90
Time:                        01:24:24   Log-Likelihood:                -130.88
No. Observations:                 100   AIC:                             271.8
Df Residuals:                      95   BIC:                             284.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.9816      0.740      4.029      0.0

In [3]:
# 1단계: 현재 모델의 p-value 확인
print("=== 1단계: 현재 모델의 유의성 검정 ===")
print("각 변수의 p-value:")
print(model.pvalues)

print("\n유의수준 0.05 기준:")
for var, pval in model.pvalues.items():
    significance = "유의함" if pval < 0.05 else "유의하지 않음"
    print(f"{var}: {pval:.6f} ({significance})")

# 2단계: 유의하지 않은 변수 식별
print("\n=== 2단계: 유의하지 않은 변수 식별 ===")
non_significant = model.pvalues[model.pvalues >= 0.05]
print(f"유의하지 않은 변수들 (p >= 0.05):")
if len(non_significant) > 0:
    for var, pval in non_significant.items():
        if var != 'Intercept':  # 절편은 제외
            print(f"  - {var}: {pval:.6f}")
else:
    print("  모든 변수가 유의함")

# 유의한 변수들만 추출 (절편 제외)
significant_vars = []
for var, pval in model.pvalues.items():
    if var != 'Intercept' and pval < 0.05:
        significant_vars.append(var)

print(f"\n유의한 변수들: {significant_vars}")

# 3단계: 새로운 회귀식 만들기
print("\n=== 3단계: 새로운 모델 생성 ===")
if len(significant_vars) == 0:
    print("유의한 변수가 없습니다!")
elif len(significant_vars) == len(model.pvalues) - 1:  # 절편 제외한 모든 변수가 유의함
    print("모든 변수가 유의하므로 기존 모델을 유지합니다.")
    new_model = model
else:
    # 새로운 회귀식 생성
    formula = f"y ~ {' + '.join(significant_vars)}"
    print(f"새로운 회귀식: {formula}")
    
    # 새 모델 적합
    from statsmodels.formula.api import ols
    new_model = ols(formula, data=df).fit()
    print(f"\n새로운 모델 요약:")
    print(new_model.summary())

# 4단계: 새 모델에서 가장 작은 회귀계수 찾기
print("\n=== 4단계: 가장 작은 회귀계수 찾기 ===")
# 절편 제외한 회귀계수들
coeffs_without_intercept = new_model.params.drop('Intercept')
print("절편 제외한 회귀계수들:")
print(coeffs_without_intercept)

# 가장 작은 값과 변수명
min_coef = coeffs_without_intercept.min()
min_var = coeffs_without_intercept.idxmin()

print(f"\n가장 작은 회귀계수:")
print(f"변수명: {min_var}")
print(f"계수값: {min_coef:.4f}")

=== 1단계: 현재 모델의 유의성 검정 ===
각 변수의 p-value:
Intercept    1.129685e-04
x1           2.100964e-81
x2           3.186170e-65
x3           1.688189e-24
x4           7.956683e-01
dtype: float64

유의수준 0.05 기준:
Intercept: 0.000113 (유의함)
x1: 0.000000 (유의함)
x2: 0.000000 (유의함)
x3: 0.000000 (유의함)
x4: 0.795668 (유의하지 않음)

=== 2단계: 유의하지 않은 변수 식별 ===
유의하지 않은 변수들 (p >= 0.05):
  - x4: 0.795668

유의한 변수들: ['x1', 'x2', 'x3']

=== 3단계: 새로운 모델 생성 ===
새로운 회귀식: y ~ x1 + x2 + x3

새로운 모델 요약:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     2672.
Date:                Fri, 20 Jun 2025   Prob (F-statistic):           2.55e-92
Time:                        01:24:25   Log-Likelihood:                -130.91
No. Observations:                 100   AIC:             

In [4]:
# 새로운 모델에서 R-squared
print("결정계수(R-squared):", new_model.rsquared)

결정계수(R-squared): 0.9881643592045125


In [5]:
# 1번 모델에서 새로운 데이터에 대한 값 예측
new_data = pd.DataFrame({"x1": [5], "x2": [12],"x3":[10],"x4":[3]})
result = model.predict(new_data)
print("새로운 데이터의 y:", result[0])

새로운 데이터의 y: -0.2433084582017353


In [6]:
# 1번 모델에서 상관관계를 구하고 가장 큰 상관계수 출력
import numpy as np

# 상관계수 행렬 계산
corr = df.corr()
print("상관계수 행렬:")
print(corr)

# 대각선을 NaN으로 만들어서 자기 상관관계 제거
corr_no_diag = corr.copy()
np.fill_diagonal(corr_no_diag.values, np.nan)

# 최댓값 구하기
max_corr = corr_no_diag.max().max()
print(f"\n자기 상관관계 제외한 최대 상관계수: {max_corr:.4f}")

상관계수 행렬:
          x1        x2        x3        x4         y
x1  1.000000 -0.115152  0.080456  0.117670  0.822594
x2 -0.115152  1.000000 -0.164049 -0.224881 -0.627445
x3  0.080456 -0.164049  1.000000  0.075652  0.301941
x4  0.117670 -0.224881  0.075652  1.000000  0.218884
y   0.822594 -0.627445  0.301941  0.218884  1.000000

자기 상관관계 제외한 최대 상관계수: 0.8226


In [7]:
# x1과 x2만을 예측 변수로 사용하는 모델을 적합하고, 
# 전체 모델과 R-squared 값을 구하시오.
from statsmodels.formula.api import ols
model_final=ols('y~x1+x2',data=df).fit()
print(model.summary())
print('--------------')
print(model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     1984.
Date:                Fri, 20 Jun 2025   Prob (F-statistic):           1.39e-90
Time:                        01:24:25   Log-Likelihood:                -130.88
No. Observations:                 100   AIC:                             271.8
Df Residuals:                      95   BIC:                             284.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.9816      0.740      4.029      0.0

In [8]:
# 잔차(residual) 분석을 수행하고, 잔차의 표준편차를 구하시오.
df['잔차'] = df['y'] - model_final.predict(df)
print(df['잔차'].std())

1.5646848719294195


In [9]:
# 1번 모델에서 새로운 데이터(x1=5, x2=12, x3=10, x4=3)에 대해 y의 신뢰구간 하한(97% 신뢰수준)을 구하세요.
new_data = pd.DataFrame({"x1": [5], "x2": [12],"x3":[10],"x4":[3]})
pred = model.get_prediction(new_data)
result = pred.summary_frame(alpha=0.03)
print("예측값의 신뢰구간과 예측구간:\n", result['mean_ci_lower'])

예측값의 신뢰구간과 예측구간:
 0   -0.540231
Name: mean_ci_lower, dtype: float64


In [10]:
# 1번 모델에서 새로운 데이터(x1=5, x2=12, x3=10, x4=3)에 대해 y의 예측구간 상한(97% 신뢰수준)을 구하세요.
new_data = pd.DataFrame({"x1": [5], "x2": [12],"x3":[10],"x4":[3]})
pred = model.get_prediction(new_data)
result = pred.summary_frame(alpha=0.03)
print("예측값의 신뢰구간과 예측구간:\n", result['mean_ci_upper'])

예측값의 신뢰구간과 예측구간:
 0    0.053614
Name: mean_ci_upper, dtype: float64
